# Stochastic Gradient Descent
[Chi tiết](https://scikit-learn.org/stable/modules/sgd.html#sgd)

Linear fitting model (SVM, logistic regression, linear regression, etc.) (config by `loss` parameter) with SGD training: tối thiểu gradient (độ dốc) hàm loss liên tục qua mỗi batch liên tục (mặc định SGD __batchsize__ = Online SGD batchsize = 1, Mini-batch SGD batchsize > 1 but < n_samples)

- Sử dụng hàm loss và fit lần lượt dữ liệu để tối thiểu hoá hàm convex loss thông qua gradient descent (đạo hàm)
- SGD fit minibatch với method `partial_fit` (chỉ có ở 1 số algorithms) cho phép học online hoặc out-of-core learning (học tăng cường/cải thiện hơn so với model trước đó)
- Nên sử dụng `StandardScaler` engineering trước khi fit
- Phù hợp với dữ xliệu lớn > 10000 obs

**Stopping criterion**
- With `early_stopping`=`True`: input data được chia thành train + validation. Model  sẽ fit train set và tiêu chí dừng lại dựa vào prediction score của validation set. size của validation set theo param `validation_fraction`
- With `early_stopping`=`False`: model sẽ fit toàn bộ input data và điều kiện dừng phụ thuộc vào objective funciton được tính toán trên tập train

Trong cả hai TH của `early_stopping`, hàm loss đều được tính toán lại sau mỗi 1 epoch, điều kiện dừng là khi 1 trong 2 TH:
- hàm loss mới có xu hướng tăng lên so với best loss (the min loss) 1 giá trị lớn hơn `tol` trong vòng `n_iter_no_change` epoch liên tiếp (tức là loss > best_loss + `tol`)
- số epoch chạm ngưỡng `max_iter`

**Pros:**
- Hiệu quả nếu tunning chính xác và tỉ mẩn
- Dễ dể thực hiện và áp dụng thực tế do có cơ chế `partial_fit`

**Cons:**
- SGD yêu cầu tunning nhiều parameters.
- SGD nhạy cảm với feature scale (need to `StandardScaler`).
- Do việc học từng phần có thêm phần chi phí lưu trữ nên chỉ sử dụng khi dữ liệu thực sự lớn và chia theo chunks

## SGD Regression

**`loss` parameter**
- loss="`squared_error`": Ordinary least squares,
- loss="`huber`": Huber loss for robust regression,
- loss="`epsilon_insensitive`": linear Support Vector Regression. (linearSVM)

In [72]:
from sklearn.linear_model import SGDRegressor

reg = SGDRegressor(loss="squared_error")
reg.fit(X_train, y_train)


# R-squared score
reg.score(X_test, y_test)

0.9494588558075366

In [73]:
# partial_fit
reg.partial_fit(X_test, y_test)

# R-squared score
reg.score(X_test, y_test)

0.950614663855244

## SGD Classification

**`loss` parameter**
- ‘`hinge`’ gives a linear SVM.
- ‘`log_loss`’ gives logistic regression, a probabilistic classifier.
- ‘`modified_huber`’ is another smooth loss that brings tolerance to outliers as well as probability estimates.
- ‘`squared_hinge`’ is like hinge but is quadratically penalized.
- ‘`perceptron`’ is the linear loss used by the perceptron algorithm.
- The other losses, ‘`squared_error`’, ‘`huber`’, ‘`epsilon_insensitive`’ and ‘`squared_epsilon_insensitive`’ are designed for regression but can be useful in classification as well; see `SGDRegressor` for a description.

**`learning_rate`**
- ‘constant’: eta = eta0
- ‘optimal’: eta = 1.0 / (alpha * (t + t0)) where t0 is chosen by a heuristic proposed by Leon Bottou.
- ‘invscaling’: eta = eta0 / pow(t, power_t)
- ‘adaptive’: eta = eta0, miễn là loss tiếp tục theo chiều hướng giảm. Nếu trong vòng `n_iter_no_change` epoch mà loss tăng vượt quá `tol` hoặc validation score bị giảm quá `tol`( trong TH set `early_stopping`=`True`) thì eta = eta/5

In [99]:
X_cl, y_cl = make_classification(n_samples=2000, n_features=20, n_informative=10, n_redundant=5, n_classes=2,flip_y = 0.1)

X_train, X_test, y_train, y_test = train_test_split(X_cl, y_cl)

In [112]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import make_pipeline

sgd = SGDClassifier(loss='hinge', # type of loss function
                    penalty='l2', # type of penalty to coef, L1 dẫn tới phần lớn các coef = 0, elasticnet cân bằng l1 + l2
                    alpha=0.0001, # hệ số nhân với penalty, càng cao thì regularization càng lớn
                    l1_ratio=0.15, # hệ số cân bằng l1 với l2 khi sử dụng elasticnet regularization
                    fit_intercept=True, # có estimate intercept hay không ?
                    max_iter=1000, # số each tối đa cho fitting (chỉ áp dụng cho fit, ko áp dụng cho partial_fit)
                    tol=0.001, # chêch lệch tối đa cho phép giữa loss hiện tại và best_loss (the min loss), nếu vượt thì sẽ dừng lại fitting
                    shuffle=True, # data có được shuffle sau mỗi 1 each được train
                    verbose=0, 
                    epsilon=0.1, # chỉ ảnh hưởng tới hàm loss của ‘huber’, ‘epsilon_insensitive’, or ‘squared_epsilon_insensitive’.
                                    # đối với 'huber' là ngưỡng mà tại đó những dự đoán chính xác trở nên ít quan tâm, chỉ care những dụ đoán sai gây ra nhiều loss
                                    # đối với ‘epsilon_insensitive’ là ngưỡng mà khi sự khác biệt giũa prediction và real mà nhỏ hơn ngưỡng này thì bỏ qua
                    n_jobs=None, 
                    random_state=None, # nếu không cố định, SGD performance bị ảnh hưởng rất nhiều bởi điểm khởi đầu
                    learning_rate='optimal', # xem tại note learning_rate
                    eta0=0.0, # learning_rate ban đầu
                    power_t=0.5, # hệ số chỉnh khi set learning_rate = 'invscaling'
                    early_stopping=False,  # có split ra validation set và xét early stopping trên đó hay không ?(stop khi validation score is not improved)
                    validation_fraction=0.1, # tỷ lệ phân chia cho validation set
                    n_iter_no_change=5, # số lượng epoch cho phép loss/validation score không được cải thiện
                    class_weight=None, # đánh trọng số cho tường class, dùng trong TH imbalance data
                    warm_start=False, # nếu True, sử dụng các tham số đã có của lần call fit trước đó làm các giá trị ban đầu, nếu ko thì xét lại từ đầu
                    average=False # coef được tính average weight cho các lần cập nhật hay ko, nếu True là all, nếu = 10 thì lấy average coef_ của 10 lần update gần nhất
                   )

clf = make_pipeline(StandardScaler(),sgd)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.684